### En este notebook vamos a ver las diferentes herramientas que hemos aprendido para evaluar el rendimiento del clasificador y la selección de aquel modelo con el mayor poder predictivo.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

In [ ]:
from sklearn import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
path_url = "https://raw.githubusercontent.com/jrasero/curso-scikit-ehu-2019/master/datasets/pima-indians-diabetes.csv"
data = pd.read_csv(path_url)

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data = data.iloc[:, 1:]

In [ ]:
data.columns = ["pregnancies", "glucose", "blood_pressure", 
              "skin_thickness","insulin","bmi","Diabetes_Pedigree_Function",
              "age","outcome"]
data.head()

In [ ]:
# Creamos nuestra matriz features y vector de targets
X = data.dropna().drop(columns=["outcome"]).values
y = data.dropna().outcome.values

### Metodo holdout

In [ ]:
# Vamos a usar un Árbol de decision. 
# Primero, creamos un objeto de la clase de este clasificador

from sklearn.tree import DecisionTreeClassifier

clf= DecisionTreeClassifier(class_weight='balanced',
                           random_state=0)
# Ajustamos los datos
clf.fit(X, y)

# Predecimos sobre los mismos datos que hemos usado para ajustar
print(clf.score(X,y))

Este resultado anterior es irreal y nada generalizable, ya que hemos usado para la predicción el mismo dataset del ajuste. Como hemos visto, para ver la generalización del modelo, tenemos que dejar una parte de la dataset fuera del ajuste (**Método holdout**)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=0)

In [ ]:
clf.fit(X_train, y_train)
print(" El rendimiento sobre el training:" ,clf.score(X_train,y_train))
print(" El rendimiento sobre el test:", clf.score(X_test,y_test))

¿Cómo dependen los resultados de los parámetros del algoritmo y el tamaño de la partición?

In [ ]:
train_scores=[]
test_scores=[]

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=0)

max_depth = np.arange(2,10)
for depth in max_depth:
    
    clf = DecisionTreeClassifier(max_depth=depth, 
                                 class_weight='balanced',
                                 random_state=0)

    clf.fit(X_train, y_train)

    train_scores.append(clf.score(X_train,y_train))
    test_scores.append(clf.score(X_test,y_test))
    
    
plt.plot(np.asarray(train_scores))
plt.plot(np.asarray(test_scores))
plt.show()

Podríamos hacer lo mismo usando la funcionalidad `validation_curve` de scikit

In [ ]:
from sklearn.model_selection import validation_curve
train_scores, test_scores = validation_curve(
    DecisionTreeClassifier(class_weight='balanced',
                           random_state=0), X, y, param_name="max_depth", 
    param_range=max_depth,
    cv=10, scoring="accuracy", n_jobs=1)

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

In [ ]:
plt.grid()

plt.fill_between(max_depth, 
                 train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.1,
                 color="r")
plt.fill_between(max_depth, 
                 test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.1, color="g")
plt.plot(max_depth, train_scores_mean, 'o-', color="r",
         label="Training score")
plt.plot(max_depth, test_scores_mean, 'o-', color="g",
         label="Cross-validation score")

plt.legend(loc="best")

Podríamos estar interesados en ver cómo varía nuestro modelo con el tamaño de nuestra dataset. Se usa para esto la funcionalidad `learning_curve` de scikit

In [ ]:
from sklearn.model_selection import learning_curve
train_sizes , train_scores, test_scores = learning_curve(
    DecisionTreeClassifier(max_depth=5,
                           class_weight='balanced',
                           random_state=0), 
    X, y, 
    train_sizes=np.linspace(.1, 1.0, 5),
    cv=10, scoring="accuracy", n_jobs=1)
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

In [ ]:
plt.grid()

plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.1,
                 color="r")
plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.1, color="g")
plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
         label="Training score")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
         label="Cross-validation score")

plt.legend(loc="best")

### Cross-validation

In [ ]:
# Vamos a definir un cross-validation con 10 folds estratificados
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10, random_state=0)

# Veamos cómo es cada fold
print('{} {}'.format('Training set observations', 'Testing set observations'))
for train_index, test_index in skf.split(X, y):                                     
    print('Num obs training: {0}, con {1} de la calse negativa y {2} de la clase positiva. Num obs en test es: {3}'.format(len(train_index), 
                                                    sum(y[train_index]==0),
                                                    sum(y[train_index]==1), 
                                                    len(test_index)))

In [ ]:
from sklearn.model_selection import cross_val_score

clf= DecisionTreeClassifier(class_weight='balanced',
                           random_state=0)

scores = cross_val_score(clf, X, y, cv=skf, 
                         scoring='accuracy')
print(scores)
print(np.mean(scores))

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score

y_pred = cross_val_predict(clf, X, y, cv=skf)
print(accuracy_score(y, y_pred))

## Mejoras a cross-validation

**Repeated cross-validation**

- Repetir cross-validation varias veces ( con **differentes particiones aleatorias** de los datos) y promediar los resultados
- Estimaciones más fiables puesto que se **reduce la variabiliad** asociada con un solo intento de cross-validation


In [ ]:
from sklearn.model_selection import RepeatedKFold
rcv = RepeatedKFold(n_splits=10, n_repeats=5, random_state=0)
clf = DecisionTreeClassifier(class_weight='balanced',
                           random_state=0)
scores = cross_val_score(clf, X, y, cv=rcv, scoring='accuracy')
print(len(scores))
print(np.mean(scores))

In [ ]:
depth_scores = []
rcv = RepeatedKFold(n_splits=10, n_repeats=5, random_state=0)
for depth in max_depth:
    clf = DecisionTreeClassifier(max_depth=depth,
                                 class_weight='balanced',
                                 random_state=0)    
    
    scores = cross_val_score(clf, X, y, cv=rcv, scoring='accuracy')
    depth_scores.append(np.mean(scores))

print(depth_scores)

In [ ]:
plt.plot(max_depth, depth_scores)
plt.xlabel('Value of depth for Decision Tree')
plt.ylabel('Cross-Validated Accuracy')
pass

## Exploración de los hiperparámetros del clasificador

Ya hemos visto que en scikit esto se puede hacer mediante la clase `GridSearchCV` y `RandomizedSearchCV`

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Creamos un grid con los parámetros sobre los que probar el clasificador
# El grid es siempre un diccionario
param_grid = {'max_depth':max_depth}
print(param_grid)

In [ ]:
# Definimos un objeto de la clase GridSearchCV y le pasamos
# el clasificador, el grid, un esquema de cross-validation y 
# un score que queremos mirar como medida de optimización
rcv = RepeatedKFold(n_splits=10, n_repeats=5, random_state=0)
grid = GridSearchCV(clf, param_grid, cv=rcv, scoring='accuracy')

In [ ]:
# Ajustamos los datos, que internamente incorpora un cross-validation
grid.fit(X, y)

In [ ]:
# Los resultados quedan almacenados en cv_results_
pd.DataFrame(grid.cv_results_)

In [ ]:
plt.plot(max_depth, grid.cv_results_['mean_test_score'])
plt.xlabel('Value of depth for Decision Tree')
plt.ylabel('Cross-Validated Accuracy')
pass

In [ ]:
# Podemos mirar las características del mejor modelo
print(grid.best_score_)
print(grid.best_params_)

### Buscando varios parámetros simultáneamente

In [ ]:
# Definimos  otra vez los hiperparámetos
max_depth = np.arange(2,10)
criterion = ['gini', 'entropy']

In [ ]:
# Ahora nuestro grid está formado por dos parámetros
param_grid = {'max_depth': max_depth, 
              'criterion': criterion}
print(param_grid)

In [ ]:
# Definimos un objeto de la clase GridSearchCV
grid = GridSearchCV(clf, param_grid, cv=rcv, scoring='accuracy')
grid.fit(X,y)

In [ ]:
pd.DataFrame(grid.cv_results_).loc[:,['params','mean_test_score']]

In [ ]:
# Podemos mirar las características del mejor modelo
print(grid.best_score_)
print(grid.best_params_)

### `RandomizedSearchCV` para reducir la carga computacional

- Buscar sobre todas las posibles combinaciones de los diferentes hiperparámetros puede ser muy costoso computacionalmente hablando.
- `RandomizedSearchCV` coge un subset de éstos en tantas iteracciones como uno desee

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
param_dist = {'max_depth': max_depth, 
              'criterion': criterion}

In [ ]:
# n_iter controla el número de busquedas sobre los parámetros
rand = RandomizedSearchCV(clf, param_dist, cv=rcv, 
                          scoring='accuracy', n_iter=10, random_state=10)
rand.fit(X, y)

In [ ]:
pd.DataFrame(rand.cv_results_)

In [ ]:
# Podemos mirar las características del mejor modelo
print(rand.best_score_)
print(rand.best_params_)